In [163]:
# !pip install -q plotly tqdm

In [164]:
# from google.colab import drive
# import sys


# drive.mount('/content/drive')
# sys.path.append("/content/drive/MyDrive/second_year_semester_B/Cognitive_of_Animal/collective-motion")


In [165]:
from tools import *

In [166]:
import timeit
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import pandas as pd
import numpy as np

import tqdm.notebook as tqdm



In [167]:
from typing import Tuple, Callable, Union, List, Optional, Any

Opt = Optional
NumType = Union[int, float]
TensorType = torch.Tensor
BoxDimType = Tuple[NumType, NumType]
UpdateRatioType = Tuple[float,...]
IndScalarType = Union[int, TensorType]
UpdateFuncType = Callable[[TensorType,dict], TensorType] 
IndexsType = Union[TensorType, List[IndScalarType], Tuple[IndScalarType, ...], range]
OptimazerType = Union[optim.Optimizer, optim.Adam, optim.SGD, optim.RMSprop]
LossType = Union[nn.Module, nn.CrossEntropyLoss, nn.MSELoss, nn.BCELoss]

<div dir="rtl" lang="he" xml:lang="he">

# מודלים טובים ומודלים טובים יותר


כאשר אנחנו חושבים על מודלים בקוגניציה, אנחנו חושבים על מודלים שמסבירים תופעה במונחים של קוגניציה. 
לדוגמה מודל 
_vicsek_
מסביר את בעלי החיים נעים בלהקה, ע"י מערכת קשרים בן הצורך לשמור על מרחק מהשני, הצורך להתאים את המהירות למהירות של השכנים וכו'.
הבעיה במודלים כאלה היא שהם כמעט לא ניתנים לבדיקה, הרעש גדול מידי. 
אם ננסה לבדוק את ה
_loss_ 
של פונק

תנועה קולקטיבית היא אחד התופעות המענייונות ביותר,
קבוצה של בעלי חיים נעה במרחב בצורה כאילו מאורגנת ללא שוב מנהיג נראה לעין.
אלו מודלים יכולים להסביר את התנועה הזאת?
והאם נמצא דרך להשוות בן המודלים האלו?

<div dir="rtl" lang="he" xml:lang="he">

## הדאטה איתו נעבוד
נשתמש בדאטה של תנועת זגי זברה.
זה יאפשר לנו לבדוק את התנועה במישור, ולא ב
3D
, במרחב
שכמו שזה קורה במציאות. 

In [168]:
# data is a tensor of shape (t,m,2)
# where t is the number of time steps and m is the number of objects.
# The last dim is (x,y) cordination in the plane.
data = torch.load("dataset/60/1.pt")
# using the cordination we can add for every point the speed, acceleration, and angle.
# so now data is a tensor of shape (t,m,5) where for every point we have (x,y, speed, acceleration,angle)
data = add_parameters(data)
# now we can plot the data, we will plot the first 40 steps.
fig = plot_timeline_with_direction(data, "zebra fish 1", steps=range(40))
fig.show()

<div dir="rtl" lang="he" xml:lang="he">

יש סדר מסויים בתנועה, אבל היא מורעשת מאוד, 
ננסה לראות מה מודל תנועה היה נותר למצב כזה. 

<div dir="rtl" lang="he" xml:lang="he">

## מודל פשוט - vicsek
המודל הבסיסי ביותר, הוא המודל של
vicsek
כל בעל חיים מתואר ע"י מיקומו,
מסומן ב
$\vec{r}_i \in R^2$
ווקטור הכיוון שלו שמסומן ב
$\vec{v}_i \in R^1$
כאשר כלל העידכון הוא הליכה בכיון הממוצע.
ע"י חישוב וקטור הכיון של השכנים, מתואר ע"י המשוואה
$$
\\ \vec{S}_i = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}_i}{|\vec{S}_i|}*v_0
$$
כאשר
$v_0$
הוא גודל קבוע המייצג את מהירות בעל החיים.




$$
\\ \vec{S} = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}}{|\vec{S}|}*v_0
$$

In [169]:
def vicsek_update(
    data: TensorType,
    parametes={
        "radius": 500,
        "update_ratio_angle": 0.7,
        "update_ratio_speed": 0.7,
        "update_ratio_acceleratoin": 0.7,
    },
) -> TensorType:
    """
    update the data according to vicsek model

    Args:
        data (TensorType): tensor (N,5) of x,y,z,theta,phi
        radius (float, optional): radius which one neighbors.Defaults to 500. in pixsels.
        parametes (dict, optional): {radius, update_ratio_angle, update_ratio_speed, update_ratio_acceleratoin} 
        of the update for angle, speed, and. Defaults to (500, 0.7, 0.7, 0.7).

    Returns:
        TensorType: tensor (N,5) new data
    """
    (
        radius,
        update_ratio_angle,
        update_ratio_speed,
        update_ratio_acceleratoin,
    ) = parametes.values()
    # get the neighbors matrix, is matrix (N,N) where each row is the neighbors of the data point
    # so data[neighbors[i]] is the neighbors of data[i]
    neighbors = get_index_neighbors(data[:, :2], radius)
    # get the avrage angle of the neighbors of each data point
    avrage_angle = torch.stack(
        [
            update_ratio_angle * data[neighbors[i], 4].mean(dim=0)
            + (1 - update_ratio_angle) * data[i, 4]
            for i in range(len(data))
        ]
    )
    # get the avrage velocity of the neighbors of each data point
    avrage_speed = torch.stack(
        [
            update_ratio_speed * data[neighbors[i], 2].mean(dim=0)
            + (1 - update_ratio_speed) * data[i, 2]
            for i in range(len(data))
        ]
    )
    avrage_acceleration = torch.stack(
        [
            update_ratio_acceleratoin * data[neighbors[i], 3].mean(dim=0)
            + (1 - update_ratio_acceleratoin) * data[i, 3]
            for i in range(len(data))
        ]
    )

    # update the coordinates
    new_x = avrage_speed * torch.cos(avrage_angle) + data[:, 0]
    new_y = avrage_speed * torch.sin(avrage_angle) + data[:, 1]
    return torch.column_stack(
        [new_x, new_y, avrage_speed, avrage_acceleration, avrage_angle]
    )

In [170]:
# create a timeline for data, so data[t] is represent the time t given by f(data[t-1])
ret = create_timeline_series(
    data[:1],
    vicsek_update,
    100,
    {
        "radius": 100,
        "update_ratio_angle": 0.7,
        "update_ratio_speed": 0.7,
        "update_ratio_acceleratoin": 0.7,
    },
)

In [171]:
plot_timeline_with_direction(ret, "test").show()

<div dir="rtl" lang="he" xml:lang="he">

ניתן לראות שהמודל הזה לא מספיק, וברור שבטווח הרחוק הוא לא יצליח לחזות את ההתנהגות, 
בעיקר שאין לנו בתוך המודל את הזירה, זה שהזירה עגולה משפיע על הסיבוב של הדגים. 


במקום לבחון את ההתהגות לטווח רחוק ננסה למצוא את הפרמטרים שחוזים את הצעד הבא בצורה הטובה ביותר. 

<div dir="rtl" lang="he" xml:lang="he">

## למידת הפרמטרים של המודל ע"י מחשב
הבעיה בצורה הנוכחית לא ניתנת ללמידה. 
(לפחות לפי הידוע לי...)
ננסה ללמוד אותה בכוח גס, כלומר ניתן רשימת פרמטרים ונבדוק מי מהם עם התוצאה הטובה ביותר. 

<div dir="rtl" lang="he" xml:lang="he">

### פונקציית הערכה
דבר ראשון צריך להגדיר פונקציית הערכה שתגדיר מתי התוצאה טובה ומתי לא. 
נשתמש בפונקציית מרחק ריבועי פשוטה. 
זאת פונקציה שרגישה למרחקים (עבור מרחקים קטנים מ1 היא קטנהמהערך הלינארי, ועבור מרחקים גדולים מ1 היא גדולה. במקרה שלנו מכיון שלא נירמנלנו את הערכים כל הערכים גדולים מ1)

In [172]:
def loss_function(pred: TensorType, target) -> TensorType:
    return nn.MSELoss()(pred, target) / target.shape[0]

<div dir="rtl" lang="he" xml:lang="he">

### פרמטרים לבדיקה
לכל אחד מהפרמטרים הבאים נגדיר ערכים רלוונטיים לבדיקה. כל ערך יופיע בפעם אחת בלבד בכל סעיף.

- המקדם של המהירות
$\alpha$
- המקדם של הזווית
$\beta$
- המקדם של התאוצה 
$\gamma$
- הרדיוס של השכנים
$r$

נעשה מכפלה קרטזית של כל הערכים ונבדוק בכל אחד מהם מה הערך הטוב ביותר.


In [173]:
from itertools import product

speed = [0.1, 0.5, 1.0, 2.0, 5.0]
angle = [0.1, 0.5, 1.0, 2.0, 5.0]
acceleratoin = [0.1, 0.5, 1.0, 2.0, 5.0]
radius = [100, 200, 300, 400, 500, 1000]
#
all_options = list(product(speed, angle, acceleratoin, radius))

In [174]:
# define dataframes to store the results
run_data = pd.DataFrame(
    columns=["speed", "angel", "acceleratoin", "radius", "loss"],
    index=range(len(all_options)),
    dtype=float,
)

<div dir="rtl" lang="he" xml:lang="he">

נריץ את הקוד על כל מבנה של אפשרויות ונראה מי מהם מביא את התוצאה הטובה ביותר

In [ ]:
batch_size = 100
for ind, optoin in tqdm.tqdm_notebook(enumerate(all_options)):
    # choice list of random indexes to evaluate
    indexes = torch.randint(0, data.shape[0] - 1, (batch_size,))
    parameters = {
        "update_ratio_speed": optoin[0],
        "update_ratio_angle": optoin[1],
        "update_ratio_acceleratoin": optoin[2],
        "radius": optoin[3],
    }
    y_pred = evaluat_in_indexes(data, indexes, vicsek_update, parametes=parameters)
    y_true = data[indexes + 1]
    y_pred = torch.nan_to_num_(y_pred, nan=0.0)
    y_true = torch.nan_to_num_(y_true, nan=0.0)
    loss = loss_function(y_pred, y_true)
    run_data.iloc[ind] = [*parameters.values(), loss.item()]
    if ind % 10 == 0:
        print(run_data.iloc[run_data["loss"].argmin()])

In [176]:
run_data.iloc[run_data["loss"].argmin()]

speed               1.000000
angel               1.000000
acceleratoin        2.000000
radius            100.000000
loss            15533.788139
Name: 378, dtype: float64

In [180]:
px.histogram(run_data["loss"]).show()

<div dir="rtl" lang="he" xml:lang="he">

ניתן לראות שאין הבדל גדול בן הפרמטרים. מקבלים מינימום על 

<div dir="rtl" lang="he" xml:lang="he">

## מודלים יותר מסובכים

<div dir="rtl" lang="he" xml:lang="he">

## השוואה בן המודלים ע"י מחשב